# STRUCTS (STRUCTURES)

*Custom __value__ data type; lightweight version of classes, but doesn't require the creation of new instances to use them*

## EXAMPLES:
* Basic numeric types (`int`, `long`, `float`, etc.)
* Basic game development geometry (x,y points, rectangle, color, etc.)
* Game development objects (player, enemies, physics, etc.)
* DateTime manipulation
* Monetary/financial manipulation
* IoT sensor data (temperatures, accelerator data, etc.)
* Custom configuration settings in your app/game
* Mock test samples

<br>

## GENERAL CONSIDERATIONS:
* Structs are used/passed in by value rather than reference (unlike classes)
* Structs are not allowed to inherit from classes nor other structs, only interfaces
* Fields CANNOT have initializers, e.g.: `private string _color = "Blue";`
* Structs CANNOT have a custom parameter-less constructor (__BEFORE C# 10__)
* If a custom constructor is written, ALL parameters must be initialized in the constructor
* Classes can reference itself or create same type instances within itself (since each instance is in a different place in memory)
    * Structs cannot because it will create a never-ending recursive definition

<br>

## GOOD PRACTICE CONSIDERATIONS:
* Use structs for small/simple data-centric types that provide little to no behavior
    * Use classes for more involved/complex object types that focus on behavior
* All of the possible values for an instantiated struct have the same hash code using the `GetHashCode()` method
    * This increases the likelihood of causing hash collisions, making systems insecure
        * Not a huge deal unless you decide to use structs for stuff that needs to be secure (e.g. passwords)
        * In that case, you would need to implement a more complex hashing algorithm
    * UNLESS you create custom `Equals()` and `GetHashCode()` method implementations in the struct
* In addition to the above method implementations, you might also need to implement operator methods for `==` and `!=`
* It is NOT good practice to use referene types inside struct implementations; use only primitive types
* It's good practice to use the `readonly` modifier for structs to make it immutable upon instantiation

<br>

## MEMORY/PERFORMANCE CONSIDERATIONS:
* Class instances are stored on the heap, whereas structs are *mostly* stored on the stack
    * However in some cases, structs can also be stored on the heap as well (especially if they are contained/used inside of a class)
* An array of instantiated structs are all stored in the same place/reference in heap memory
* Classes relative to structs: each instantiated class object, even if they are contained inside an array, are still each located in different places in heap memory
    * The array containing class instances will only contain pointer references to the different locations of each instance in heap memory
* Structs cannot be `null`, they will automatically have a place allocated in memory for it regardless of whether it's being used or not
    * Classes on the other hand can have a pointer reference to `null` until the instance is initalized and a reference is established using the `new` keyword
* Can be efficient when it comes to performance, but only if the instantiated struct is under 16 bytes in size
    * Since structs are passed in by value rather than reference, structs will need to be copied and will be performance-intensive if the struct is too big
    * e.g GUIDs (globally unique identifiers) are expensive to use as a struct since they are exactly 16 bytes in size
    * Therefore, it's best to stick to simple implementations when using structs

In [5]:
struct Car
{
    public string Color { get; set; }

    public Car(string color)
    {
        this.Color = color;
    }

    public string Describe()
    {
        return "This car is " + Color;
    }
}


Car car;

car = new Car("Blue");
Console.WriteLine(car.Describe());

car = new Car("Red");
Console.WriteLine(car.Describe());

This car is Blue
This car is Red


In [19]:
struct Student
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public int GradeLevel { get; set; }

    public Student(string firstname, string lastname)
    {
        this.FirstName = firstname;
        this.LastName = lastname;
    }

    public string ShowStudentInfo()
    {
        StringBuilder info = new StringBuilder();

        info.AppendLine("First Name\tLast Name\tGrade Level");
        info.AppendLine($"{this.FirstName}\t\t{this.LastName}\t\t{this.GradeLevel}");

        return info.ToString();
    }

    public string NextGradeLevel()
    {
        // '?' in case grade level is NULL for Student type
        int? currentGradeLevel = this.GradeLevel;
        int? nextGradeLevel = currentGradeLevel + 1;
        return $"Next Grade Level: {nextGradeLevel}";
    }
}

Student s1 = new Student("Jonny", "Johnson");
s1.GradeLevel = 5;
Console.WriteLine(s1.ShowStudentInfo());
Console.WriteLine(s1.NextGradeLevel());

Console.WriteLine("\n");

Student s2 = new Student("Rebecca", "Beck");
s2.GradeLevel = 11;
Console.WriteLine(s2.ShowStudentInfo());
Console.WriteLine(s2.NextGradeLevel());

First Name	Last Name	Grade Level
Jonny		Johnson		5

Next Grade Level: 6


First Name	Last Name	Grade Level
Rebecca		Beck		11

Next Grade Level: 12


### Custom Operator Implementations

In [35]:
struct Point
{
    public int X { get; set; }
    public int Y { get; set; }

    public Point(int x, int y)
    {
        this.X = x;
        this.Y = y;
    }

    public override string ToString()
    {
        return $"({this.X}, {this.Y})";
    }

    // what to do when comparing points using `==`
    public static bool operator ==(Point p1, Point p2)
    {
        return p1.X == p2.X && p1.Y == p2.Y;
    }

    // what to do when comparing points using `!=`
    public static bool operator !=(Point p1, Point p2)
    {
        return p1.X != p2.X || p1.Y != p2.Y;
    }

    // what to do when comparing points using `Point.Equals()` (should do the same thing as `==`; value equality)
    public override bool Equals(object obj)
    {
        if (!(obj is Point))
        {
            return false;
        }
        return obj is Point p && this.X == p.X && this.Y == p.Y;
    }


    public override int GetHashCode()
    {
        return (this.X, this.Y).GetHashCode();
    }
}

Point p1 = new Point(0, 0);
Point p2 = new Point(10, 20);
Point p3 = new Point(0, 0);
Point p4 = p1;

Console.WriteLine($"Does {p1}['p1']   ==   {p2}['p2']: {p1 == p2}");
Console.WriteLine($"Does {p1}['p1']   !=   {p2}['p2']: {p1 != p2}");
Console.WriteLine($"Does {p1}['p1']   ==   {p3}['p3']: {p1 == p3}");
Console.WriteLine($"Does {p1}['p1']   !=   {p3}['p3']: {p1 != p3}");
Console.WriteLine($"Does {p1}['p1'].Equals({p3}['p3']): {p1.Equals(p3)}");
Console.WriteLine($"Does {p1}['p1']   ==   {p4}['p4']: {p1 == p4}");
Console.WriteLine($"Does {p1}['p1'].Equals({p4}['p4']): {p1.Equals(p4)}");

Does (0, 0)['p1']   ==   (10, 20)['p2']: False
Does (0, 0)['p1']   !=   (10, 20)['p2']: True
Does (0, 0)['p1']   ==   (0, 0)['p3']: True
Does (0, 0)['p1']   !=   (0, 0)['p3']: False
Does (0, 0)['p1'].Equals((0, 0)['p3']): True
Does (0, 0)['p1']   ==   (0, 0)['p4']: True
Does (0, 0)['p1'].Equals((0, 0)['p4']): True
